# MATPOWER IEEE Case 9

R.P. Schulz, A.E. Turner and D.N. Ewart, "Long Term Power System Dynamics," EPRI Report 90-7-0, Palo Alto, California, 1974.

In [1]:
%load_ext autoreload
%autoreload 2

## DC power flow with fixed voltage

In [2]:
import os
import numpy as np

from GridCal.Engine import Bus, Load, Line, Generator, MultiCircuit
from GridCal.Engine import PowerFlowOptions, PowerFlowDriver, SolverType
from GridCal.Engine.IO import parse_matpower_file

from matpower import path_matpower

import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

Bentayga is not available
Newton native is not available


In [3]:
def round_power_close_to_zero(power):
    real_power = np.real(power)
    real_power[np.isclose(real_power,0,atol=1e-06)] = 0

    imag_power = np.imag(power)
    imag_power[np.isclose(imag_power,0,atol=1e-06)] = 0

    return real_power, imag_power

In [4]:
case_name = 'case9.m'
case_path = os.path.join(path_matpower, 'data', case_name)

grid, _ = parse_matpower_file(case_path)

In [5]:
for bus in grid.buses:
    bus.is_dc = True
    for load in bus.loads:
        load.Q = 0
    if bus.controlled_generators:
        bus.is_slack = True

for line in grid.lines:
    line.X = 0
    line.B = 0
    if line.R == 0:
        line.R = 0.001

In [6]:
options = PowerFlowOptions(SolverType.NR, verbose=False)
power_flow = PowerFlowDriver(grid, options)
power_flow.run()

In [7]:
print('|V|:', abs(power_flow.results.voltage))
print('|Sbranch|:', abs(power_flow.results.Sf))
print('|loading|:', abs(power_flow.results.loading) * 100)
print('err:', power_flow.results.error)
print('Conv:', power_flow.results.converged)
print('Total real losses:', sum(np.real(power_flow.results.losses)))
print('Power flow:', power_flow.results.If)

|V|: [1.04       1.025      1.025      1.03790057 1.02343881 1.02455797 1.01963934 1.02446231 1.02541321]
|Sbranch|: [218.34052835  88.29334579   2.9369042   45.30778169  42.34812712  57.85517519  55.08457954   3.04425492 128.04708058]
|loading|: [87.33621134 35.31733832  1.95793613 15.1025939  28.23208474 23.14207008 22.03383182  1.21770197 51.21883223]
err: 4.923249828103403e-09
Conv: True
Total real losses: 3.7618014326236118
Power flow: [ 2.09942816+0.j  0.85069175+0.j -0.02869643+0.j  0.44202714+0.j  0.41333071+0.j -0.56740823+0.j -0.53769259+0.j -0.02971564+0.j -1.24873641+0.j]


In [8]:
df_bus, df_branch = power_flow.results.export_all()
df_branch['Imag power (MVAr)'] = abs(df_branch['Imag power (MVAr)'])
df_branch

,Real power (MW),Imag power (MVAr),Power module (MVA),Loading(%),Losses (MW),Losses (MVAr),Losses (MVA),Tap module
0,218.341,0.000,218.341,0.873,0.441,0.000,0.441,NaN
1,88.293,0.000,88.293,0.353,1.230,0.000,1.230,NaN
2,-2.937,0.000,2.937,0.020,0.003,0.000,0.003,NaN
3,45.308,0.000,45.308,0.151,0.020,0.000,0.020,NaN
4,42.348,0.000,42.348,0.282,0.203,0.000,0.203,NaN
5,-57.855,0.000,57.855,0.231,0.274,0.000,0.274,NaN
6,-55.085,0.000,55.085,0.220,0.029,0.000,0.029,NaN
7,-3.044,0.000,3.044,0.012,0.003,0.000,0.003,NaN
8,-128.047,0.000,128.047,0.512,1.559,0.000,1.559,NaN


In [9]:
df_bus['Real power (MW)'], df_bus['Imag power (MVAr)'] = round_power_close_to_zero(power_flow.results.Sbus)
df_bus


,Real voltage (p.u.),Imag Voltage (p.u.),Voltage module (p.u.),Voltage angle (rad),Real power (MW),Imag power (MVAr)
0,1.040,0.000,1.040,0.000,218.341,0.000
1,1.025,0.000,1.025,0.000,55.113,0.000
2,1.025,0.000,1.025,0.000,45.308,0.000
3,1.038,0.000,1.038,0.000,0.000,0.000
4,1.023,0.000,1.023,0.000,-90.000,0.000
5,1.025,0.000,1.025,0.000,0.000,0.000
6,1.020,0.000,1.020,0.000,-100.000,0.000
7,1.024,0.000,1.024,0.000,0.000,0.000
8,1.025,0.000,1.025,0.000,-125.000,0.000


## DC power flow with fixed voltage and limited power

This section is not fully developed since fixed voltage can't work together with limited power. My workaround is to change bus type to fixed power if power hit PMIN or PMAX during achieving fixed voltage in limited power, and re-run the simulation.

In [10]:
from GridCal.Engine import StaticGenerator

In [11]:
grid.buses[1].controlled_generators[0].Pmax = 30

In [12]:
for bus, bus_power in zip(grid.buses, df_bus['Real power (MW)'].values):
    if bus.controlled_generators:
        # NOTE: This approach only support one generator for each bus
        sum_p_max = sum(generator.Pmax for generator in bus.controlled_generators)
        sum_p_min = sum(generator.Pmin for generator in bus.controlled_generators)
        if sum_p_min <= bus_power <= sum_p_max:
            continue
        
        bus.is_slack = False
        
        if bus_power > sum_p_max:
            # add imaginary static generator
            static_generator = StaticGenerator(
                P=sum_p_max,
                Q=0,
            )
        elif bus_power < sum_p_min:
            # add imaginary static generator
            static_generator = StaticGenerator(
                P=sum_p_min,
                Q=0,
            )
        
        bus.static_generators.append(static_generator)

        for controlled_generator in bus.controlled_generators:
            controlled_generator.active = False

In [17]:
options = PowerFlowOptions(SolverType.NR, verbose=False)
power_flow = PowerFlowDriver(grid, options)
power_flow.run()

In [18]:
print('|V|:', abs(power_flow.results.voltage))
print('|Sbranch|:', abs(power_flow.results.Sf))
print('|loading|:', abs(power_flow.results.loading) * 100)
print('err:', power_flow.results.error)
print('Conv:', power_flow.results.converged)
print('Total real losses:', sum(np.real(power_flow.results.losses)))
print('Power flow:', power_flow.results.If)

|V|: [1.04       1.02124938 1.025      1.03781677 1.02332975 1.02439462 1.01751556 1.02095563 1.02450621]
|Sbranch|: [227.05624838  88.44040619   2.79414475  62.05112649  59.21742607  41.18023349  29.99137063  11.32808696 136.36748271]
|loading|: [90.82249935 35.37616248  1.86276316 20.68370883 39.47828405 16.4720934  11.99654825  4.53123478 54.54699309]
err: 4.933942995628391e-09
Conv: True
Total real losses: 4.107375377142733
Power flow: [ 2.18323316+0.j  0.85217747+0.j -0.02730444+0.j  0.60537684+0.j  0.5780724 +0.j -0.40471355+0.j -0.29375783+0.j -0.11095572+0.j -1.3310557 +0.j]


In [19]:
df_bus, df_branch = power_flow.results.export_all()
df_branch['Imag power (MVAr)'] = abs(df_branch['Imag power (MVAr)'])
df_branch

,Real power (MW),Imag power (MVAr),Power module (MVA),Loading(%),Losses (MW),Losses (MVAr),Losses (MVA),Tap module
0,227.056,0.000,227.056,0.908,0.477,0.000,0.477,NaN
1,88.440,0.000,88.440,0.354,1.235,0.000,1.235,NaN
2,-2.794,0.000,2.794,0.019,0.003,0.000,0.003,NaN
3,62.051,0.000,62.051,0.207,0.037,0.000,0.037,NaN
4,59.217,0.000,59.217,0.395,0.398,0.000,0.398,NaN
5,-41.180,0.000,41.180,0.165,0.139,0.000,0.139,NaN
6,-29.991,0.000,29.991,0.120,0.009,0.000,0.009,NaN
7,-11.328,0.000,11.328,0.045,0.039,0.000,0.039,NaN
8,-136.367,0.000,136.367,0.545,1.772,0.000,1.772,NaN


In [20]:
df_bus['Real power (MW)'], df_bus['Imag power (MVAr)'] = round_power_close_to_zero(power_flow.results.Sbus)
df_bus


,Real voltage (p.u.),Imag Voltage (p.u.),Voltage module (p.u.),Voltage angle (rad),Real power (MW),Imag power (MVAr)
0,1.040,0.000,1.040,0.000,227.056,0.000
1,1.021,0.000,1.021,0.000,30.000,0.000
2,1.025,0.000,1.025,0.000,62.051,0.000
3,1.038,0.000,1.038,0.000,0.000,0.000
4,1.023,0.000,1.023,0.000,-90.000,0.000
5,1.024,0.000,1.024,0.000,0.000,0.000
6,1.018,0.000,1.018,0.000,-100.000,0.000
7,1.021,0.000,1.021,0.000,0.000,0.000
8,1.025,0.000,1.025,0.000,-125.000,0.000
